# 必要データ出力

In [ ]:
options(warn=-1)
options(future.globals.maxSize = 100 * 1024^3)
Pkgs <- c("Matrix", "tibble", "tidyr", "dplyr", "stringr", "hdf5r", "zellkonverter", "Seurat", "SingleCellExperiment", "glmGamPoi", "magrittr", "ggplot2", "gplots", "RColorBrewer", "grid")
for(p in Pkgs) suppressMessages(library(p, character.only=T))


x <- readH5AD("data/adata_healthy_merfish.h5ad", reader="R")
write.table(x@colData[,c(1,3:10,14)], file="data/colData.tsv", sep="\t", quote=FALSE, row.names=TRUE, col.names=NA)
write.table(as.data.frame(as.matrix(assay(x, "X"))), file="data/expmat.tsv", sep="\t", quote=FALSE, row.names=TRUE, col.names=NA)

# 切片画像作成

In [ ]:
using Colors, CairoMakie
using CSV, DataFrames

Data = CSV.read("data/colData.tsv", header=1, comment="#", delim='\t', DataFrame)
CellTypes = sort(unique(Data.Cell_Type))
Palette = distinguishable_colors(length(CellTypes))
ColourMap = Dict(CellTypes[i] => Palette[i] for i in eachindex(CellTypes))
Smpl = sort(unique(Data.sample_id))
SmplNum = length(Smpl)
fig = Figure(size=(1000*SmplNum+100, 1000), fontsize=32)
ax = []
for i in 1:SmplNum
    push!(ax, CairoMakie.Axis(
                        fig[1, i];
                        xlabel = "", ylabel="", title=Smpl[i],
                        xgridvisible=false, ygridvisible=false,
                        xticklabelsvisible=false, yticklabelsvisible=false,
                        xticksvisible=false, yticksvisible=false
                    )
    )
    for ct in CellTypes
        Q = (Data.Cell_Type .== ct) .& (Data.sample_id .== Smpl[i])
        scatter!(ax[i], Data.x[Q], Data.y[Q]; color=ColourMap[ct], markersize=7, label=string(ct), strokewidth=0)
    end
end
markers = [MarkerElement(marker=:circle, color=ColourMap[ct], strokecolor=:transparent, markersize=32) for ct in CellTypes]
Legend(fig[2, 1:SmplNum], markers, CellTypes; framevisible=false, orientation=:horizontal, labelsize=32)
save("Result/Fig/tissue.pdf", fig)


# SGCP

In [ ]:
options(warn=-1)
options(future.globals.maxSize = 100 * 1024^3)
Pkgs <- c("SGCP", "org.Hs.eg.db", "AnnotationDbi", "clusterProfiler", "enrichplot", "ggplot2")
for(p in Pkgs) suppressMessages(library(p, character.only=T))

Analysis_GO <- function(x) {
        result <- enrichGO(
                                gene=x, keyType="SYMBOL", OrgDb=org.Hs.eg.db, ont="ALL", #"BP","CC","MF","ALL"
                                pAdjustMethod="BH", pvalueCutoff=0.05, qvalueCutoff=0.05
                        )
}
PlotFig <- function(res, s, i) {
        # res_simple <- simplify(res)
        # Fig <- clusterProfiler::dotplot(res_simple)
        # ggsave(paste0("Result/Fig/Visium_brain_SGCP_",s,"_GO_cluser-",i,"_dot_simple.pdf"), plot=Fig)
        Fig <- clusterProfiler::dotplot(res)
        ggsave(paste0("Result/Fig/Visium_brain_SGCP_",s,"_GO_cluser-",i,"_dot_all.pdf"), plot=Fig)
}

Data <-  read.table("data/colData.tsv", header=TRUE, row.names=NULL)
Data$row.names <- sub("-", ".", paste0("X", Data$row.names))
ExpMat <- read.table("data/expmat.tsv", header=TRUE, row.names=NULL)
names(ExpMat)[1] <- "SYMBOL"
genes <- AnnotationDbi::select(org.Hs.eg.db, keys=ExpMat$SYMBOL, columns=c("ENTREZID"), keytype="SYMBOL")
genes <- genes[(!is.na(genes$SYMBOL)) & (!is.na(genes$ENTREZID)), ]
genes <- genes[(!duplicated(genes$SYMBOL)) & (!duplicated(genes$ENTREZID)),]
ExpMat <- merge(genes, ExpMat, by="SYMBOL")
genes <- ExpMat[, 1:2]
ExpMat <- ExpMat[, -c(1:2)]
vars <- apply(ExpMat, 1, var)
zeroInd <- which(vars == 0)
if(length(zeroInd) != 0) {
        ExpMat <- ExpMat[-zeroInd, ]
        genes <- genes[-zeroInd, ]
}
xx <- as.list(org.Hs.egGO[genes$ENTREZID])
haveGO <- sapply(xx, function(x) {if (length(x) == 1 && is.na(x)) FALSE else TRUE })
numNoGO <- sum(!haveGO)
if(numNoGO != 0){
        ExpMat <- ExpMat[haveGO, ]
        genes <- genes[haveGO, ]
}
rownames(ExpMat) <- genes$ENTREZID

Smpl <- unique(Data$sample_id)
for (s in Smpl) {
        sgcp <- ezSGCP(expData=as.matrix(ExpMat[, Data$row.names[Data$sample_id == s]]), geneID=rownames(ExpMat), annotation_db="org.Hs.eg.db", eff.egs=FALSE , saveOrig=FALSE, sil=TRUE, hm=NULL)
        names(genes) <- c("Symbol","geneID")
        res <- merge(genes, sgcp$clusterLabels, by="geneID")[,c(2,4)]
        write.table(res, paste0("Result/SGCP_",s,".tsv"), sep='\t', row.names=FALSE)

        clust <- unique(res$finalClusters)
        for (c in clust) {
                commun = res$Symbol[res$finalClusters == c]
                result <- try(Analysis_GO(commun), silent=FALSE)
                if (class(result) != "try-error") {
                        y <- try(PlotFig(result, s, c), silent=FALSE)
                }
        }
}



# hdWGCNA

In [ ]:
options(warn=-1)
Pkgs <- c("zellkonverter", "Seurat", "SingleCellExperiment", "hdWGCNA", "org.Hs.eg.db", "AnnotationDbi", "clusterProfiler", "enrichplot", "ggplot2")
for(p in Pkgs) suppressMessages(library(p, character.only=T))

Analysis_GO <- function(x) {
        result <- enrichGO(
                                gene=x, keyType="SYMBOL", OrgDb=org.Hs.eg.db, ont="ALL", #"BP","CC","MF","ALL"
                                pAdjustMethod="BH", pvalueCutoff=0.05, qvalueCutoff=0.05
                        )
}
PlotFig <- function(res, s, i) {
        # res_simple <- simplify(res)
        # Fig <- clusterProfiler::dotplot(res_simple)
        # ggsave(paste0("Result/Fig/Visium_brain_SGCP_",s,"_GO_cluser-",i,"_dot_simple.pdf"), plot=Fig)
        Fig <- clusterProfiler::dotplot(res)
        ggsave(paste0("Result/Fig/Visium_brain_hdWGCNA_",s,"_GO_cluser-",i,"_dot_all.pdf"), plot=Fig)
}

# 読込
so <- readH5AD("data/adata_healthy_merfish.h5ad", reader="R")
smpl <- as.character(unique(colData(so)$sample_id))
sce_list <- setNames(
        lapply(smpl, function(s) so[, so$sample_id==s, drop=FALSE]),
        smpl
)
so_list <- lapply(names(sce_list), function(id){
        x <- sce_list[[id]]
        obj <- as.Seurat(x, counts="X", data="X")
})
names(so_list) <- names(sce_list)

so_list <- lapply(names(so_list), function(id){
        obj <- so_list[[id]]
        # Construct metaspots
        so <- SetupForWGCNA(obj, gene_select="fraction", fraction=0.05, wgcna_name="vis")
        # so <- MetaspotsByGroups(so, group.by=c("region"), ident.group="region", assay="SCT")
        # so <- NormalizeMetacells(so)

        # Co-expression network analysis
        so <- SetDatExpr(so, group.by=NULL, group_name=NULL)
        so <- TestSoftPowers(so)
        powdf <- so@misc$vis$wgcna_powerTable
        hit <- which(powdf$truncated.R.sq >= 0.8)
        best_power <- 0.8
        if (length(hit) == 0)
        {
                best_power <- powdf$Power[ which.max(powdf$truncated.R.sq) ]
        } else {
                best_power <- powdf$Power[ min(hit) ]
        }
        best_power <- as.integer(max(1, min(50, round(best_power))))
        # plot_list <- PlotSoftPowers(so)
        # wrap_plots(plot_list, ncol=2)
        so <- ConstructNetwork(so, soft_power=best_power, tom_name="test", overwrite_tom=TRUE)
        # PlotDendrogram(so, main='Spatial hdWGCNA dendrogram')

        # so <- ModuleEigengenes(so)
        # so <- ModuleConnectivity(so)
        modules <- GetModules(so)
        write.table(modules[,c(1,3)], paste0("Result/hdWGCNA_",id,".tsv"), sep='\t', row.names=FALSE, col.names=TRUE, quote=FALSE)

        clust <- unique(modules$color)
        for (c in clust) {
                commun = modules$gene_name[modules$color == c]
                result <- try(Analysis_GO(commun), silent=FALSE)
                if (class(result) != "try-error") {
                        y <- try(PlotFig(result, id, c), silent=FALSE)
                }
        }
})

# SGCRNA

In [ ]:
using CSV, DataFrames
using JLD2
using SGCRNAs
using RCall

# load data
Data = CSV.read("data/colData.tsv", header=1, comment="#", delim='\t', DataFrame);
ExpMat = CSV.read("data/expmat.tsv", header=1, comment="#", delim='\t', DataFrame);
Smpl = unique(Data.sample_id)
Gene = ExpMat.Column1
for s in Smpl
    # calc correlation and gradient
    CorData, GradData = CGM(Gene, Matrix(ExpMat[:,Data.Column1[Data.sample_id .== s]]), threshold=0.9)
    # save_object("tmp/predata.jld2", (CorData, GradData));
    # CorData, GradData = load_object("tmp/predata.jld2");
    println(s, " CGM done")
    # spectral clustering
    clust, pos, edge_data = SpectralClustering(CorData, GradData; pcas=10, nNeighbors=5);
    # save_object("tmp/scdata.jld2", (clust, pos, edge_data));
    # clust, pos, edge_data = load_object("tmp/scdata.jld2");
    println(s, " SC done")

    # # set parameter
    # for i in 1:length(clust) println(maximum(clust[i])); end
    # for i in 1:maximum(clust[1]) println(i, "-", sum(clust[1] .== i)); end

    # set parameter
    d = 1; k = maximum(clust[d]);
    DataFrame(Gene=names(edge_data), Module=clust[d]) |> CSV.write("Result/SGCRNA_"*s*".tsv", delim='\t', writeheader=true)

    # GO analysis
    # ro = RObject(edge_data);
    # new_clust = RObject(clust[1]);
    # R"""
    #     Pkgs <- c("clusterProfiler", "enrichplot", "org.Hs.eg.db", "ggplot2")
    #     for(p in Pkgs) suppressMessages(library(p, character.only=T))

    #     options(ggrepel.max.overlaps = Inf)

    #     Analysis_GO <- function(x) {
    #         result <- enrichGO(
    #                             gene=x, keyType="SYMBOL", OrgDb=org.Hs.eg.db, ont="ALL", #"BP","CC","MF","ALL"
    #                             pAdjustMethod="BH", pvalueCutoff=0.05, qvalueCutoff=0.05
    #                         )
    #     }
    #     PlotFig <- function(res, s, i) {
    #             # res_simple <- simplify(res)
    #             # Fig <- clusterProfiler::dotplot(res_simple)
    #             # ggsave(paste0("Result/Fig/Visium_brain_SGCRNA_",$s,"_GO_cluser-",i,"_dot_simple.pdf"), plot=Fig)
    #             Fig <- clusterProfiler::dotplot(res)
    #             ggsave(paste0("Result/Fig/Visium_brain_SGCRNA_",$s,"_GO_cluser-",i,"_dot_all.pdf"), plot=Fig)
    #     }

    #     for(i in 1:$k)
    #     {
    #         ed <- $ro
    #         nc <- $new_clust
    #         commun = colnames(ed)[which(nc == i)]
    #         result <- try(Analysis_GO(commun), silent=FALSE)
    #         if (class(result) != "try-error") {
    #             y <- try(PlotFig(result, $s, i), silent=FALSE)
    #         }
    #     }
    # """

    # println(s, "GO done")
end


# spatial expression heatmap of each module


In [ ]:
using Images, FileIO
using Colors, ColorSchemes, CairoMakie
using CSV, DataFrames

Data_All = CSV.read("data/colData.tsv", header=1, comment="#", delim='\t', DataFrame)
Smpl = unique(Data_All.sample_id)
SmplNum = length(Smpl)
ExpDF_main = CSV.read("data/expmat.tsv", header=true, comment="#", delim='\t', DataFrame);
rename!(ExpDF_main, :Column1 => :Gene);
ColourList = RGB.(HSV.(range(240, stop=0, length=50), 1.0, 1.0))
ColourMap = cgrad(ColourList, 100)
for t in["SGCRNA", "SGCP", "hdWGCNA"]
    for i in 1:SmplNum
        Modules = CSV.read("Result/"*t*"_"*Smpl[i]*".tsv", header=true, comment="#", delim='\t', DataFrame);
        rename!(Modules, [:Gene,:Module])
        Data = Data_All[Data_All.sample_id .== Smpl[i], :]
        Module_list = unique(Modules.Module)
        for m in Module_list
            ExpDF = innerjoin(Modules[Modules.Module .== m,[1]], ExpDF_main, on=:Gene);
            PlotData = innerjoin(Data, DataFrame(Column1=names(ExpDF)[2:end],Val=map(sum,eachcol(ExpDF[:,2:end]))), on=:Column1)

            # fig = Figure(size=(maximum(PlotData.x)-minimum(PlotData.x)+100, maximum(PlotData.y)-minimum(PlotData.y)+10), fontsize=14)
            fig = Figure(fontsize=14)
            ax = CairoMakie.Axis(
                                fig[1, 1];
                                xlabel="", ylabel="", title="",
                                xgridvisible=false, ygridvisible=false,
                                xticklabelsvisible=false, yticklabelsvisible=false,
                                xticksvisible=false, yticksvisible=false
                    )
            scatter!(ax, PlotData.x, PlotData.y; color=PlotData.Val, colormap=ColourList, markersize=3, strokewidth=0)
            Colorbar(fig[1, 2], limits=(0,maximum(map(maximum, eachcol(ExpDF[:,2:end])))), colormap=ColourMap, label="")

            save("Result/Fig/Visium_brain_"*t*"_"*Smpl[i]*"_cluster-"*string(m)*"_heatmap.pdf", fig)
        end
    end
    println(t)
end


# 要素比較ヒートマップ(対ツール)

In [ ]:
using CSV, DataFrames
using Colors, CairoMakie

Smpl = ["AM042", "AM048", "AM061"]
Tools = ["hdWGCNA", "SGCP"]
for s in Smpl
    SGCRNA = CSV.read("Result/SGCRNA_"*s*".tsv", header=1, comment="#", delim='\t', DataFrame);
    rename!(SGCRNA, [:Symbol,:Module])
    N_Sgcrna = maximum(SGCRNA.Module)
    Result = [];
    ClustName =[]
    for t in Tools
        Buf = CSV.read("Result/"*t*"_"*s*".tsv", header=1, comment="#", delim='\t', DataFrame)
        rename!(Buf, [:Symbol,:Module])
        push!(ClustName, sort(unique(Buf.Module)))
        N_Buf = length(ClustName[end])
        Result_Buf = zeros(N_Buf, N_Sgcrna)
        for c in 1:N_Sgcrna
            A = Set(SGCRNA.Symbol[SGCRNA.Module .== c])
            for c2 in 1:N_Buf
                B = Set(Buf.Symbol[Buf.Module .== ClustName[end][c2]])
                Result_Buf[c2, c] = length(A ∩ B) / min(length(A), length(B))
            end
        end
        push!(Result, Result_Buf)
    end

    fig = Figure(size=(sum([size(Result[i],1) for i in 1:length(Result)])*100, size(Result[1],2)*50), fontsize=16)
    ax = []
    for i in 1:length(Result)
        push!(ax, Axis(fig[1, i]; xlabel="", ylabel="", title=Tools[i], xgridvisible=false, ygridvisible=false, xticks=(1:size(Result[i],1), string.(ClustName[i])), xticklabelrotation=π/4, yticklabelsvisible=false, yticksvisible=false))
        heatmap!(ax[i], Result[i], colormap=:heat, colorrange=(0.0,1.0))
    end
    ax[1].ylabel = "SGCRNA"
    ax[1].yticklabelsvisible = true
    ax[1].yticksvisible = true
    ax[1].yticks = (1:size(Result[1],2), string.(1:size(Result[1],2)))
    Colorbar(fig[:, end+1], limits=(0, 1), colormap=:heat)
    colsize!(fig.layout, 1, Relative(size(Result[1],1)/(size(Result[1],1)+size(Result[2],1))))
    save("Result/Fig/ModuleSimilarity_"*s*".pdf", fig)
end


# 要素比較ヒートマップ(対試料)

In [ ]:
using CSV, DataFrames
using Colors, CairoMakie

Smpl = ["AM048", "AM061"]
Tools = ["SGCRNA", "hdWGCNA", "SGCP"]
FigSize = Dict(
            "SGCRNA" => (900,700),
            "hdWGCNA" => (300,250),
            "SGCP" => (300,250)
)
for t in Tools
    AM042 = CSV.read("Result/"*t*"_AM042.tsv", header=1, comment="#", delim='\t', DataFrame);
    rename!(AM042, [:Symbol,:Module])
    Src_Clust = sort(unique(AM042.Module))
    N_AM042 = length(Src_Clust)
    Result = [];
    ClustName =[];
    for s in Smpl
        Buf = CSV.read("Result/"*t*"_"*s*".tsv", header=1, comment="#", delim='\t', DataFrame)
        rename!(Buf, [:Symbol,:Module])
        push!(ClustName, sort(unique(Buf.Module)))
        N_Buf = length(ClustName[end])
        Result_Buf = zeros(N_Buf, N_AM042)
        for c in 1:N_AM042
            A = Set(AM042.Symbol[AM042.Module .== Src_Clust[c]])
            for c2 in 1:N_Buf
                B = Set(Buf.Symbol[Buf.Module .== ClustName[end][c2]])
                Result_Buf[c2, c] = length(A ∩ B) / min(length(A), length(B))
            end
        end
        push!(Result, Result_Buf)
    end

    fig = Figure(size=FigSize[t], fontsize=16)
    ax = []
    for i in 1:length(Result)
        push!(ax, Axis(fig[1, i]; xlabel="", ylabel="", title=Smpl[i], xgridvisible=false, ygridvisible=false, xticks=(1:size(Result[i],1), string.(ClustName[i])), xticklabelrotation=π/4, yticklabelsvisible=false, yticksvisible=false))
        heatmap!(ax[i], Result[i], colormap=:heat, colorrange=(0.0,1.0))
    end
    ax[1].ylabel = "AM042"
    ax[1].yticklabelsvisible = true
    ax[1].yticksvisible = true
    ax[1].yticks = (1:size(Result[1],2), string.(Src_Clust))
    Colorbar(fig[:, end+1], limits=(0, 1), colormap=:heat)
    colsize!(fig.layout, 1, Relative(size(Result[1],1)/(size(Result[1],1)+size(Result[2],1))))
    save("Result/Fig/ModuleSimilarity_"*t*".pdf", fig)
end
